In [2]:
import argparse
import os
from data_utils.S3DISDataLoader import RhobanDataset
import torch
import datetime
import logging
from pathlib import Path
import sys
import importlib
import shutil
from tqdm import tqdm
import provider
import numpy as np
import time

BASE_DIR = '/home/starkit/Pointnet_Pointnet2_pytorch'
ROOT_DIR = BASE_DIR
sys.path.append(os.path.join(ROOT_DIR, 'models'))

In [3]:
log_dir = '2024-05-05_15-38'
experiment_dir = Path('./log/')
experiment_dir.mkdir(exist_ok=True)
experiment_dir = experiment_dir.joinpath('sem_seg')
experiment_dir.mkdir(exist_ok=True)
experiment_dir.mkdir(exist_ok=True)
checkpoints_dir = experiment_dir.joinpath('checkpoints/')
checkpoints_dir.mkdir(exist_ok=True)
log_dir = experiment_dir.joinpath('logs/')
log_dir.mkdir(exist_ok=True)


In [5]:
NUM_CLASSES = 2

In [6]:
def inplace_relu(m):
    classname = m.__class__.__name__
    if classname.find('ReLU') != -1:
        m.inplace=True

In [8]:
model = 'pointnet_sem_seg'
MODEL = importlib.import_module(model)
shutil.copy('models/%s.py' % model, str(experiment_dir))
shutil.copy('models/pointnet2_utils.py', str(experiment_dir))

classifier = MODEL.get_model(NUM_CLASSES).cuda()
criterion = MODEL.get_loss().cuda()
classifier.apply(inplace_relu)
print(classifier)

get_model(
  (feat): PointNetEncoder(
    (stn): STN3d(
      (conv1): Conv1d(9, 64, kernel_size=(1,), stride=(1,))
      (conv2): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
      (conv3): Conv1d(128, 1024, kernel_size=(1,), stride=(1,))
      (fc1): Linear(in_features=1024, out_features=512, bias=True)
      (fc2): Linear(in_features=512, out_features=256, bias=True)
      (fc3): Linear(in_features=256, out_features=9, bias=True)
      (relu): ReLU(inplace=True)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn3): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn5): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv1): Conv1d(9, 64, kernel_size=(1,), stride=

In [10]:
list(classifier.named_parameters())

[('feat.stn.conv1.weight',
  Parameter containing:
  tensor([[[ 0.1363],
           [-0.2732],
           [-0.2481],
           [-0.0937],
           [-0.2412],
           [ 0.2155],
           [ 0.0527],
           [-0.2519],
           [-0.1668]],
  
          [[ 0.0408],
           [ 0.2802],
           [-0.3085],
           [-0.0902],
           [ 0.2041],
           [-0.1163],
           [-0.0608],
           [-0.1746],
           [ 0.0521]],
  
          [[-0.1335],
           [ 0.2979],
           [-0.0864],
           [-0.1749],
           [-0.1340],
           [-0.1837],
           [ 0.0296],
           [ 0.2427],
           [-0.0095]],
  
          [[ 0.2781],
           [ 0.3057],
           [ 0.2042],
           [ 0.2636],
           [ 0.2083],
           [ 0.2107],
           [ 0.0406],
           [ 0.2090],
           [-0.0638]],
  
          [[ 0.1870],
           [-0.0286],
           [ 0.0595],
           [ 0.2534],
           [ 0.0245],
           [-0.2619],
         

In [13]:
names = ['conv1.weight', 'conv1.bias', 'conv2.weight', 'conv2.bias', 'conv3.weight', 'conv3.bias','conv4.weight', 'conv4.bias','bn1.weight', 'bn1.bias','bn2.weight', 'bn2.bias', 'bn3.weight', 'bn3.bias',]
for param in classifier.named_parameters():
    print(dir(param))
    param.requires_grad = False

TypeError: 'generator' object is not subscriptable

In [12]:
list(classifier.named_parameters())

[('feat.stn.conv1.weight',
  Parameter containing:
  tensor([[[ 0.1363],
           [-0.2732],
           [-0.2481],
           [-0.0937],
           [-0.2412],
           [ 0.2155],
           [ 0.0527],
           [-0.2519],
           [-0.1668]],
  
          [[ 0.0408],
           [ 0.2802],
           [-0.3085],
           [-0.0902],
           [ 0.2041],
           [-0.1163],
           [-0.0608],
           [-0.1746],
           [ 0.0521]],
  
          [[-0.1335],
           [ 0.2979],
           [-0.0864],
           [-0.1749],
           [-0.1340],
           [-0.1837],
           [ 0.0296],
           [ 0.2427],
           [-0.0095]],
  
          [[ 0.2781],
           [ 0.3057],
           [ 0.2042],
           [ 0.2636],
           [ 0.2083],
           [ 0.2107],
           [ 0.0406],
           [ 0.2090],
           [-0.0638]],
  
          [[ 0.1870],
           [-0.0286],
           [ 0.0595],
           [ 0.2534],
           [ 0.0245],
           [-0.2619],
         